In [ ]:
# Imports

import socket
from emoji import demojize
import pandas as pd
import re
from datetime import datetime, timedelta
from collections import Counter

In [ ]:
# Constants, hide these
# In order for this to work, use your twitch username for nickname and get a token at https://twitchapps.com/tmi/

server = 'irc.chat.twitch.tv'
port = 6667
channel = 'symfuhny'
nickname = ''
token = ''
data = []
delta = timedelta(minutes = 1)
sentiment_df = pd.read_csv('sentiment_scores.csv', index_col = "word")

In [ ]:
# Method to extract important information from twitch calls and write them to data frame
# format for regex reference :larocke!larocke@larocke.tmi.twitch.tv PRIVMSG #larocke :8

def add_chat_dataframe(line):
    now = datetime.now()
    try:

        username, channel, message = re.search(':(.*)\!.*@.*\.tmi\.twitch\.tv PRIVMSG #(.*) :(.*)', line).groups()

        d = {
            'dt': now,
            'channel': channel,
            'username': username,
            'message': message
        }

        data.append(d)
            
    except Exception:
        pass
    
    # Once a comment is added anything older than time delta is deleted. Will only work for big stream.
    
    for i in data:
        if i['dt'] < now - delta:
            data.remove(i)

In [ ]:
# parses through words, and finds their sentiment scores which are added to the return value

def get_sentiment_score(words):
    result = 0
    
    for word in words:
        try:
            result += sentiment_df.loc[word].sentiment.item()
        except:
            pass
    
    return result

In [ ]:
# splits current dataframe into words and finds most common one and calls for sentiment score

def get_info():
    words = []
    for i in data:
        split_sentence = i['message'].split()
        for j in split_sentence:
            words.append(j)
        
    top_one = Counter(words).most_common(1)
    sentiment_score = get_sentiment_score(words)
    
    print(top_one, sentiment_score)

In [ ]:
# Imports Initializations

s = socket.socket()
s.connect((server, port))

In [ ]:
# Sending oatharization info to twitch

s.send(f"PASS {token}\n".encode('utf-8'))
s.send(f"NICK {nickname}\n".encode('utf-8'))
s.send(f"JOIN #{channel}\n".encode('utf-8'))

In [ ]:
resp = s.recv(2048).decode('utf-8').lower()

In [ ]:
# Main loop that monitors chat, writes to dataframe and calls for overall sentiment

while True:
    resp = s.recv(2048).decode('utf-8').lower()

    if resp.startswith('PING'):
        s.send("PONG\n".encode('utf-8'))
    
    elif len(resp) > 0:
        add_chat_dataframe(demojize(resp))
        get_info()
        